In [1]:
import csv
import ipyparallel as ipp
import timeit

In [33]:
with open('500.testbedtools.processed', newline = '') as eccloc:
    eccloc_reader = csv.reader(eccloc, delimiter = '\t')
    eccloc_list = [[int(row[0][10:12]), int(row[1]), int(row[2])] for row in eccloc_reader]
    
with open('discordantmappedreads.oppositefacing.bed', newline = '') as discordant:
    discordant_reader = csv.reader(discordant, delimiter = '\t')
    discordant_list = [[int(row[0][10:12]), int(row[1]), int(row[2])] for row in discordant_reader]

In [35]:
print(discordant_list[0:10])

[[6, 10395, 10529], [6, 10666, 10817], [26, 97376, 97494], [26, 96398, 96549], [21, 62575, 62726], [21, 62009, 62160], [3, 2591710, 2591861], [3, 2591451, 2591602], [16, 108349, 108500], [16, 106839, 106990]]


576688


In [4]:
import sys
sys.getsizeof(discordant_list)

68742480

In [36]:
mydict = dict(discordant_list = discordant_list)

In [12]:
mydict2 = dict(eccloc_list = eccloc_list)

In [13]:
print(mydict2)

{'eccloc_list': [['MQOP01000016.1', '576686', '576764', 'A00887:30:HNJ7FDSXX:4:1101:10149:27649/1', '60', '-'], ['MQOP01000015.1', '70735', '71303', 'A00887:30:HNJ7FDSXX:4:1101:10167:7106/1', '60', '-'], ['MQOP01000015.1', '349128', '349764', 'A00887:30:HNJ7FDSXX:4:1101:10203:25488/1', '60', '-'], ['MQOP01000003.1', '1674894', '1680235', 'A00887:30:HNJ7FDSXX:4:1101:10285:35931/1', '60', '-'], ['MQOP01000013.1', '355104', '356042', 'A00887:30:HNJ7FDSXX:4:1101:10294:7200/1', '60', '-'], ['MQOP01000036.1', '7036', '7186', 'A00887:30:HNJ7FDSXX:4:1101:10529:23923/1', '0', '-'], ['MQOP01000001.1', '3581517', '3581848', 'A00887:30:HNJ7FDSXX:4:1101:1127:8140/1', '60', '-'], ['MQOP01000003.1', '626071', '627316', 'A00887:30:HNJ7FDSXX:4:1101:11586:30420/1', '60', '-'], ['MQOP01000008.1', '2156466', '2157043', 'A00887:30:HNJ7FDSXX:4:1101:11668:22200/1', '60', '-'], ['MQOP01000022.1', '209465', '209961', 'A00887:30:HNJ7FDSXX:4:1101:12002:30514/1', '60', '-'], ['MQOP01000005.1', '2696604', '2699034

In [8]:
sys.getsizeof(mydict)

240

In [37]:
def get_size(obj, seen=None):

    """Recursively finds size of objects"""

    size = sys.getsizeof(obj)

    if seen is None:

        seen = set()

    obj_id = id(obj)

    if obj_id in seen:

        return 0

    # Important mark as seen *before* entering recursion to gracefully handle

    # self-referential objects

    seen.add(obj_id)

    if isinstance(obj, dict):

        size += sum([get_size(v, seen) for v in obj.values()])

        size += sum([get_size(k, seen) for k in obj.keys()])

    elif hasattr(obj, '__dict__'):

        size += get_size(obj.__dict__, seen)

    elif hasattr(obj, '__iter__') and not isinstance(obj, (str, bytes, bytearray)):

        size += sum([get_size(i, seen) for i in obj])

    return size

In [39]:
import sys
print(get_size(mydict))

1207280944


In [14]:
print(get_size(discordant_list))

3287952224


In [9]:
print(mydict)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [44]:
def confirmeccs(ecc):
    for i in range(0, len(discordant_list), 2):
        read1 = discordant_list[i]
        read2 = discordant_list[i+1]
        if read1[0]==ecc[0]==read2[0]:
            if ecc[1] <= read1[1] <= ecc[2] and ecc[1] <= read1[2] <= ecc[2] and ecc[1] <= read2[1] <= ecc[2] and ecc[1] <= read2[2] <= ecc[2]:
                return True
    return False

In [45]:
def confirmeccs_old(ecc):
    f = 0
    for i in range(0, len(discordant_list), 2):
        read1 = discordant_list[i]
        read2 = discordant_list[i+1]
        if read1[0]==ecc[0]==read2[0]:
            if ecc[1] <= read1[1] <= ecc[2] and ecc[1] <= read1[2] <= ecc[2] and ecc[1] <= read2[1] <= ecc[2] and ecc[1] <= read2[2] <= ecc[2]:
                f += 1
                break
    if f > 0:
        return True
    else:
        return False

In [48]:
yesornoeccs = list(map(confirmeccs, eccloc_list))

In [49]:
yesornoeccs_old = list(map(confirmeccs_old, eccloc_list))

In [50]:
yesornoeccs == yesornoeccs_old

True

In [43]:
print(yesornoeccs)

[False, True, True, True, True, False, False, True, False, True, True, False, True, False, True, True, True, False, False, False, True, False, True, True, False, False, True, True, True, True, False, False, True, False, True, False, True, False, True, True, True, True, False, False, False, True, False, False, True, False, True, True, True, True, False, False, True, True, True, True, True, True, False, False, True, True, True, False, True, False, True, True, True, False, True, True, True, True, True, False, False, False, True, True, True, True, True, True, False, False, False, True, True, True, False, True, True, True, True, True, True, False, True, True, True, False, True, True, False, True, True, True, False, True, True, True, False, True, True, True, True, False, True, True, True, False, False, True, True, False, True, True, True, True, False, False, False, False, False, True, False, False, False, False, False, True, True, False, True, False, True, False, False, True, True, True, Tru

In [45]:
from itertools import compress

In [49]:
confirmedeccs = list(compress(eccloc_list, yesornoeccs))

In [4]:
rc = ipp.Client(profile='default', cluster_id='')

In [12]:
rc.ids # prints names of cores

[9]

In [7]:
mydict = dict(discordant_list = discordant_list)

In [8]:
dview = rc[:] # refers to all cores
dview.block = True # master has to wait for workers

In [9]:
dview.push(mydict)

<AsyncResult: _push>

In [11]:
confirmedeccs = dview.map(confirmeccs, eccloc_list)

In [7]:
lview = rc.load_balanced_view() #load balanced view
lview.block = True

In [ ]:
yesornoeccs = lview.map(confirmeccs, eccloc_list)

In [22]:
start_time = timeit.default_timer()
confirmedeccs = confirmeccs(eccloc_list)
print(timeit.default_timer() - start_time)

In [50]:
with open('parallel.confirmed', 'w', newline = '') as confirmed:
    w = csv.writer(confirmed, delimiter = '\t')
    w.writerows(confirmedeccs)

In [51]:
! head 'parallel.confirmed'

In [52]:
! head 'writeatend.confirmed'